In [2]:
# !pip install biopython
# !pip install findspark
# !pip install py4j

In [9]:
from graphframes import GraphFrame
from pyspark.sql.functions import coalesce, col, lit, sum, when, max, collect_list, min, udf, struct
from pyspark.sql.types import *
from graphframes.lib import Pregel

In [8]:
def vertexProgram(vd: Row, msg:Row):
    print('Vertex', vd, msg, sep=': ')
#     if msg == None: # or float(vd[0]) < float(msg[0]):
#         return (vd[0] , vd[1] , vd[2])
#     else:
#         return (msg[0] , vd[1] , msg[2])


vertexProgramUdf = udf(vertexProgram)

def sendMsgToDst(src:Row, dst:Row):
#     srcDist = float(src[0] )
#     dstDist = float(dst[0] )
    print('Send', src, dst, sep=': ')
#     if srcDist < (dstDist - 1):
#         return (srcDist + 1, src[1] , src[2] + ' ' + dst[1])
#     else:
#         return None

sendMsgToDstUdf = udf(sendMsgToDst)

def aggMsgs(agg: [Row]):
    return (agg[0] [0] , agg[0] [1] , agg[0] [2])


aggMsgsUdf = udf(aggMsgs)

In [9]:
# edges = spark.createDataFrame([[2, 1, 1],[1, 2, 1],[2, 4, 1],[3, 6, 1],[4, 1, 1], [5, 2, 1], [5, 3, 1], [5, 6, 1]], ["src", "dst", 'attr'])
# edges.cache()
# vertices = spark.createDataFrame([[0], [1], [2], [3], [4], [5], [6]], ["id"])
# numVertices = vertices.count()
# vertices = GraphFrame(vertices, edges).outDegrees
# vertices.cache()
# graph = GraphFrame(vertices, edges)
# # alpha = 0.15
# sc.setCheckpointDir("/tmp/graphframes-example-connected-components")
# root_node = 0
# schema = StructType([
#     StructField("dist", DoubleType(), True),
#     StructField("name", StringType(), True),
#     StructField("path", StringType(), True)
# ])
# ranks = graph.pregel \
#     .setMaxIter(3) \
#     .withVertexColumn("path", when(col('id')==(lit(root_node)), struct(lit(0), col('id'), col('id').astype(StringType()))) \
#                       .otherwise(struct(lit(10000), col('id'), lit(''))).cast(schema) \
#                       , vertexProgramUdf(col('path'), Pregel.msg())) \
#     .sendMsgToDst(sendMsgToDstUdf(Pregel.src("path"), Pregel.dst("path"))) \
#     .aggMsgs(aggMsgsUdf(collect_list(Pregel.msg()))) \
#     .run()

KeyboardInterrupt: 

GraphFrame Pregel

In [5]:
edges = spark.createDataFrame([[0, 1], [1, 2],[2, 4],[2, 0],[3, 4],[4, 2]], ["src", "dst"])
edges.cache()
vertices = spark.createDataFrame([[0], [1], [2], [3], [4]], ["id"])
numVertices = vertices.count()
vertices = GraphFrame(vertices, edges).outDegrees
vertices.cache()
graph = GraphFrame(vertices, edges)
alpha = 0.15

2022-11-22 11:43:54 NumExpr defaulting to 4 threads.


In [21]:
def vertexProgram(msg):
    print('Vertex','Msg', msg, 'Msg type', type(msg), sep=': ')
    return (msg if msg != None else 0) * (1.0 - alpha) + alpha / numVertices


vertexProgramUdf = udf(vertexProgram)

def sendMsgToDst(srcRank, srcOutDegree):
    print('Send','SrcRank', srcRank, 'SrcRank type', type(srcRank), 'SrcOutDegree', srcOutDegree, 'SrcOutDegree type', type(srcOutDegree), sep=': ')
    return float(srcRank) / srcOutDegree


sendMsgToDstUdf = udf(sendMsgToDst)

def aggMsgs(msg: [Row]):
    return sum(msg)


aggMsgsUdf = udf(aggMsgs)

In [22]:
sc.setCheckpointDir("/tmp/graphframes-example-connected-components")
ranks = graph.pregel \
    .setMaxIter(5) \
    .withVertexColumn("rank", lit(1.0 / numVertices), vertexProgramUdf(Pregel.msg())) \
    .sendMsgToDst(sendMsgToDstUdf(Pregel.src("rank"),Pregel.src("outDegree"))) \
    .aggMsgs(aggMsgs(Pregel.msg())) \
    .run()

In [23]:
ranks.show()

+---+---------+-------------------+
| id|outDegree|               rank|
+---+---------+-------------------+
|  0|        1|0.19903352343750003|
|  1|        1|      0.17089228125|
|  3|        1|               0.03|
|  2|        2|0.37554067187499995|
|  4|        1|0.22453352343750002|
+---+---------+-------------------+



In [24]:
ranks = graph.pregel \
 .setMaxIter(5) \
  .withVertexColumn("rank", lit(1.0 / numVertices), \
      coalesce(Pregel.msg(), lit(0.0)) * lit(1.0 - alpha) + lit(alpha / numVertices)) \
 .sendMsgToDst(Pregel.src("rank") / Pregel.src("outDegree")) \
   .aggMsgs(sum(Pregel.msg())) \
   .run()

In [25]:
ranks.show()

+---+---------+-------------------+
| id|outDegree|               rank|
+---+---------+-------------------+
|  0|        1|0.19903352343750003|
|  1|        1|      0.17089228125|
|  3|        1|               0.03|
|  2|        2|0.37554067187499995|
|  4|        1|0.22453352343750002|
+---+---------+-------------------+



In [1]:
from pyspark.sql import *
from pyspark import *
from pyspark.accumulators import AccumulatorParam
from Bio import SeqIO
import networkx as nx

In [2]:
import logging
import re
import itertools as it
import gc
import sys
import findspark
findspark.init()
from collections import namedtuple
date_strftime_format = '%Y-%m-%y %H:%M:%S'
logging.basicConfig(stream=sys.stdout, level=logging.INFO, format="%(asctime)s %(message)s", datefmt=date_strftime_format)

In [3]:
class Log4j:
    def __init__(self, spark):
        root_class = "guru.learningjournal.spark.examples"
        conf = spark.sparkContext.getConf()
        app_name = conf.get("spark.app.name")
        log4j = spark._jvm.org.apache.log4j
        self.logger = log4j.LogManager.getLogger(root_class + '.' + app_name)
    
    def warn(self, message):
        self.logger.warn(message)
        logging.warn(message)
        
    def info(self, message):
        self.logger.info(message)
        logging.info(message)
    
    def error(self, message):
        self.logger.error(message)
        logging.error(message)
    
    def debug(self, message):
        self.logger.debug(message)
        logging.debug(message)

Initial SparkSession

In [4]:
spark = SparkSession.builder.appName('Hello Spark').master('local[*]').getOrCreate()
sc = spark.sparkContext
sc.addPyFile('/Users/DELL/.ivy2/jars/graphframes_graphframes-0.8.2-spark3.1-s_2.12.jar')
logger = Log4j(spark)
from graphframes import *

In [7]:
def load_meta_reads(filename, type='fasta'):
    try:
        seqs = list(SeqIO.parse(filename, type))

        reads = []
        labels = []

        # Detect for paired-end or single-end reads
        # If the id of two first reads are different (e.g.: .1 and .2), they are paired-end reads
        is_paired_end = False
        if len(seqs) > 2 and seqs[0].id[-1:] != seqs[1].id[-1:]:
            is_paired_end = True

        label_list = dict()
        label_index = 0
        for i in range(0, len(seqs), 2 if is_paired_end else 1):
            read, label = format_read(seqs[i])
            if is_paired_end:
                read2, label2 = format_read(seqs[i + 1])
                read += read2
            reads += [str(read)]
        
            # Create labels
            if label not in label_list:
                label_list[label] = label_index
                label_index += 1
            labels.append(label_list[label])
        
        del seqs
        return reads, labels
    except:
        print('Error when loading file {} '.format(filename))
        return []

In [8]:
def format_read(read):
    # Return sequence and label
    z = re.split('[|={,]+', read.description)
    return read.seq, z[3]

In [9]:
reads, labels = load_meta_reads('data/S1.fna', type='fasta')

In [10]:
L_MER = 20

In [11]:
class DictParam(AccumulatorParam):
    def zero(self,  value = ""):
        return dict()

    def addInPlace(self, value1, value2):
        for i in value2.keys():
            if i in value1:
                value1[i].append(value2[i])
            else:
                value1[i] = [value2[i]]
        return value1

In [12]:
class DictEdgeParam(AccumulatorParam):
    def zero(self,  value = ""):
        return dict()

    def addInPlace(self, value1, value2):
        for i in value2.keys():
            if i in value1:
                value1[i] += value2[i]
            else:
                value1[i] = value2[i]
        return value1

In [13]:
def build_dict_origin():
    logging.info('Start 1')
    lmers_dict = dict()
    for idx, r in enumerate(reads):
        for j in range(0,len(r)-L_MER+1):
            lmer = r[j:j+L_MER]
            if lmer in lmers_dict:
                lmers_dict[lmer] += [idx]
            else:
                lmers_dict[lmer] = [idx]
    E=dict()
    for lmer in lmers_dict:
        for e in it.combinations(lmers_dict[lmer],2):
            if e[0]!=e[1]:
                e_curr=(e[0],e[1])
                if e_curr in E:
                    E[e_curr] += 1 # Number of connected lines between read a and b
                else:
                    E[e_curr] = 1
    E_Filtered = {kv[0]: kv[1] for kv in E.items() if kv[1] >= 20}
    
    G = nx.Graph()
    print('Adding nodes...')
    color_map = {0: 'red', 1: 'green', 2: 'blue', 3: 'yellow', 4: 'darkcyan', 5: 'violet',
                6: 'black', 7: 'grey', 8: 'sienna', 9: 'wheat', 10: 'olive', 11: 'lightgreen',
                12: 'cyan', 13: 'slategray', 14: 'navy', 15: 'hotpink'}
    for i in range(0, len(labels)):
        G.add_node(i, label=labels[i], color=color_map[labels[i]])

    print('Adding edges...')
    for kv in E_Filtered.items():
        G.add_edge(kv[0][0], kv[0][1], weight=kv[1])
    print('Graph constructed!')
    logging.info('End 1')
    return G
#     print(E_Filtered[(0, 29033)])
#     print(len(E_Filtered.keys()))
#     print(lmers_dict[ATAAATACCTTCATTTAATA])


In [14]:
def build_dict_spark_map(readsRDD, spark):
    logging.info('Start 2')
    def create_lmers_pos(tuple):
        idx, r = tuple
        lmers_dict =list()
        for j in range(0,len(r)-L_MER+1):
            lmer = r[j:j+L_MER]
            lmers_dict.append((lmer, idx))
#         print(lmers_dict)
        return lmers_dict
    def create_edge(x):
        lmer, idx = x
#         print(lmer, idx)
        global edge_dict
        E=dict()
        for e in it.combinations(idx,2):
            if e[0]!=e[1]:
                e_curr=(e[0],e[1])
                if e_curr in E:
                    E[e_curr] += 1 # Number of connected lines between read a and b
                else:
                    E[e_curr] = 1
        edge_dict += E
    logging.info("Building hash table...")
    lmers_dict = readsRDD.map(create_lmers_pos).flatMap(lambda x: [i for i in x]).groupByKey().mapValues(list).filter(lambda x: len(x[1]) > 2)
    logging.info('Build edge ...')
    lmers_dict.coalesce(20).foreach(create_edge)
    # Step 1: map
        # ['asdasd': 2, 'asdasdgg': 3, 'asdasd': 4]; ['asdasd': 2, 'asdasdgg': 3, 'asdasd': 4]
    # Step 3: flat
        # ['asdasd': [2, 4, 90000], 'asdasdgg': 3]
    #     res = readsRDD.map(create_lmers_pos).flatMap(lambda x: [i for i in x]).reduceByKey(lambda x,y: x.append(y)).count()
    global edge_dict
    E = edge_dict.value
    E_Filtered = {kv[0]: kv[1] for kv in E.items() if kv[1] >= 20}
    color_map = {0: 'red', 1: 'green', 2: 'blue', 3: 'yellow', 4: 'darkcyan', 5: 'violet',
            6: 'black', 7: 'grey', 8: 'sienna', 9: 'wheat', 10: 'olive', 11: 'lightgreen',
            12: 'cyan', 13: 'slategray', 14: 'navy', 15: 'hotpink'}
    logging.info('Add nodes ...')
    vertices = spark.createDataFrame([(i, labels[i], color_map[labels[i]]) for i in range(0, len(labels))], ['id', 'colorId', 'color'])
    logging.info('Add edges ...')
    edges = spark.createDataFrame([(kv[0][0], kv[0][1], kv[1]) for kv in E_Filtered.items()], ['src', 'dst', 'numOfLmers'])
    logging.info('Building graph ...')
    g = GraphFrame(vertices, edges)
    return g
    logging.info('End 2')
#     return res


In [15]:
def build_dict_spark_foreach(readsRDD):
    logging.info('Start 3')
    def create_dict_foreach(tuple):
        idx, r = tuple
        global lmers_dict_3
        for j in range(0,len(r)-L_MER+1):
            lmer = r[j:j+L_MER]
            lmers_dict_3 += {lmer: idx}
    readsRDD.foreach(create_dict_foreach)
    global lmers_dict_3
    res = lmers_dict_3.value
#     print(res['ATAATTGGCAAGTGTTTTAG'])
    print(len(res.keys()))
    logging.info('End 3')
    return res

In [16]:
def build_dict_spark_mapPartition(readsRDD):
    logging.info('Start 4')
    def create_dict_mapPartition(partitionData):
        lmers_dict = dict()
        for idx, r in [*partitionData]:
            for j in range(0,len(r)-L_MER+1):
                lmer = r[j:j+L_MER]
                if lmer in lmers_dict:
                    lmers_dict[lmer] += [idx]
                else:
                    lmers_dict[lmer] = [idx]
        yield lmers_dict
    def merge_dict(x,y):
        for i in y.keys():
            if i in x:
                x[i] += y[i] 
            else:
                x[i] = y[i]
        return x

    lmers_dict = readsRDD.mapPartitions(create_dict_mapPartition).reduce(lambda x, y: merge_dict(x,y))
    logging.warning('Processing 1')
    E=dict()
    for lmer in lmers_dict:
        for e in it.combinations(lmers_dict[lmer],2):
            if e[0]!=e[1]:
                e_curr=(e[0],e[1])
            if e_curr in E:
                E[e_curr] += 1
            else:
                E[e_curr] = 1
    E_Filtered = {kv[0]: kv[1] for kv in E.items() if kv[1] >= 20}
    
#     print(E_Filtered[(0, 29033)])
    print(len(E_Filtered.keys()))
    
#     logging.warning('Processing 2')
#     res = readsRDD.mapPartitions(create_dict_mapPartition).collect()
#     count = 0
#     for dict1 in res:
#         count += len(dict1.keys())
#     print(count)
#     logging.warning('End Processing 2')
#     def create_edge(dictionary):
#         E  = dict()
#         for lmer in dictionary:
#             for e in it.combinations(dictionary[lmer],2):
#                 if e[0]!=e[1]:
#                     e_curr=(e[0],e[1])
#                 if e_curr in E:
#                     E[e_curr] += 1 # Number of connected lines between read a and b
#                 else:
#                     E[e_curr] = 1
#         E_Filtered = {kv[0]: kv[1] for kv in E.items() if kv[1] >= 20}
#         return E_Filtered
# #     first = readsRDD.mapPartitions(create_dict_mapPartition).map(create_edge).collect()
# #     print(first)
#     res = readsRDD.mapPartitions(create_dict_mapPartition).map(create_edge).collect()
    
#     count = 0
#     for edge in res:
#         count += len(edge.keys())
#     print(count)
# #     print(res[(49008, 56213)])
    logging.info('End 4')
#     return res?

In [32]:
# def build_overlap_graph( reads, spark ):

edge_dict = spark.sparkContext.accumulator({}, DictEdgeParam())

readsRDD = spark.sparkContext.parallelize(enumerate(reads)).repartition(40).cache()

#     build_dict_origin()
# G = build_dict_spark_map(readsRDD,spark)

G.vertices.show()
G.edges.show()

logging.info('Connected Components Algorithm ...')
sc.setCheckpointDir("/tmp/graphframes-example-connected-components")
CC = G.connectedComponents()

CC.show()
    
#     lmers_dict_3 = build_dict_spark_foreach(readsRDD)
#     build_dict_spark_mapPartition(readsRDD)


+---+-------+-----+
| id|colorId|color|
+---+-------+-----+
|  0|      0|  red|
|  1|      0|  red|
|  2|      0|  red|
|  3|      0|  red|
|  4|      0|  red|
|  5|      0|  red|
|  6|      0|  red|
|  7|      0|  red|
|  8|      0|  red|
|  9|      0|  red|
| 10|      0|  red|
| 11|      0|  red|
| 12|      0|  red|
| 13|      0|  red|
| 14|      0|  red|
| 15|      0|  red|
| 16|      0|  red|
| 17|      0|  red|
| 18|      0|  red|
| 19|      0|  red|
+---+-------+-----+
only showing top 20 rows

+-----+-----+----------+
|  src|  dst|numOfLmers|
+-----+-----+----------+
|  390|34894|        46|
|  390|41175|        46|
|34894|41175|        53|
|  391|34566|        30|
|  391|40346|        30|
|34566|40346|        66|
|  392|25279|        20|
|  392|24117|        44|
|25279|24117|        73|
|  392| 5907|        40|
|24117| 5907|        34|
|  393|40975|        42|
|17347|40975|        22|
|17347| 3568|        43|
|41630|23664|        43|
|41630|10503|        41|
|23664|10503|      

In [34]:
Phân tích connected componennt 

SyntaxError: invalid syntax (2504951288.py, line 1)

CC.groupBy('component').count().orderBy('component', ascending=True).show()

In [18]:
# if __name__ == '__main__':
# #     sg = spark.createDataFrame
#     edge_dict = spark.sparkContext.accumulator({}, DictEdgeParam())
# #     lmers_dict_3 = sc.accumulator({}, DictParam())
# #     lmers_dict_4 = sc.accumulator({}, DictParam())
# #     lmers_dict = sc.accumulator({}, DictParam())
# #     gc.collect()
#     logger.info('Start')
# #     dict_test = build_overlap_graph(reads, sc)
#     build_overlap_graph(reads, spark)
#     logger.info('End')

In [19]:
# print(type(lmers_dict_3))

In [20]:
# print(dict_test['ATAAATACCTTCATTTAATA'])

In [21]:
# sc.stop()